# Mystic Stealer
> The many variants of this new stealer

- toc: true 
- badges: true
- categories: [mystic stealer,stealer,obfuscation,cpp]

## Overview

According to Zscaler Mystic is a stealer that has been active since April 2023, sold on underground forum such as XSS. Other than stealing browser credentials and crypto wallets the stealer's main differentiator is the use of a custom obfuscator that is used to protect strings. This obfuscator produces similar output as [ADVObfuscator](https://github.com/andrivet/ADVobfuscator), there as been speculation that it is indeed just ADVObfuscator.

### Samples

BF38A3699AB2072DEA806FF2EE3E54FCA4ABFA983BE9CBB207C3AE8E65095364 [UnpacMe](https://www.unpac.me/results/c5e9d9b5-9621-4f67-8a30-907509fc7ff4?hash=bf38a3699ab2072dea806ff2ee3e54fca4abfa983be9cbb207c3ae8e65095364#/)

### References
- [Mystic Stealer](https://www.zscaler.com/blogs/security-research/mystic-stealer)
- [Mystic Stealer – Evolving “stealth” Malware](https://www.cyfirma.com/outofband/mystic-stealer-evolving-stealth-malware/)


## Analysis

Looking at the C2 encryption routine we can see that there was a progression from the earlier builds. 

The earliest build from March 2023 `47439044a81b96be0bb34e544da881a393a30f0272616f52f54405b4bf288c7c` has a build path that has not been erased `G:\Projects\stealer\oGnSUE7arNOZser\tmp_compiler\output.pdb`. This build does not use the custom encryption algorithm described below but simply uses an encrypted stack string to hide the c2 `164.132.200.171`.

Builds from April-May 2023 (example `45D29AFC212F2D0BE4E198759C3C152BB8D0730BA20D46764A08503EAB0B454F`) uses the algorithm with a stack string containing the IP address as a DWORD followed by the port as a WORD and a stack string key.

New builds (example `BF38A3699AB2072DEA806FF2EE3E54FCA4ABFA983BE9CBB207C3AE8E65095364`) have used a modified version of the algorithm which embed the key in the algorithm and used a global encrypted string to hide the full URL of the C2, not just an IP and Port.


## Config

The following is a modified version of the open source decryptor for Zscaler [decrypt_c2s.py](https://github.com/threatlabz/tools/blob/main/mystic_stealer/decrypt_c2s.py)

In [1]:
import struct

def uint32(val):
    return val & 0xffffffff

def decrypt(data):
    out = b''
    block_size = 8  
    num_blocks = len(data) // block_size
    blocks = struct.unpack(f"<{2 * num_blocks}L", data)

    for i in range(0,len(blocks),2):
        out += decrypt_block(blocks[i],blocks[i+1])
    return out

def decrypt_block(v0, v1):
    sum_value = 0xC6EF3720
    delta = 0x61C88647
    
    #key0, key1, key2, key3 = struct.unpack("<4L", key)
    
    key0 = 0x7D935554
    key1 = 0x7A3B0639
    key2 = 0x4C774985
    key3 = 0x8F036C0
    
    for i in range(32):
        v1 = v1 - ((v0 + sum_value) ^ (key2 + (v0 << 4)) ^ (key3 + (v0 >> 5)))
        v1 = uint32(v1)
        # print("v1:", hex(v1))
        v7 = uint32(v1 + sum_value)
        sum_value = uint32(sum_value + delta)
        v8 = v7 ^ uint32(key0 + (v1 << 4)) ^ uint32(key1 + (v1 >> 5))
        v0 =  uint32(v0 - v8)
        
    
    return struct.pack("<2L", v0, v1)

decrypt(bytes.fromhex('6f928688f64c75a5c916e8abe4560c29e279d7b750aba83f'))

b'http://193.233.254.61/\x00\x00'

The encrypted C2 can take two forms, either an IP address in DWORD octet format, followed by a port, or a full URL. The octet format follows. 

In [21]:
c21 = 0x91736C72
c22 = 0x17DEE303
c23 = 0x85B9C50
c24 = 0xD8A3FC07

c221 = 0x66EB0028;
c222 = 0x41978887;
c223  = 0x85B9C50;
c224  = 0xD8A3FC07;


def decrypt_block(v0, v1):
    sum_value = 0xC6EF3720
    delta = 0x61C88647
    
    #key0, key1, key2, key3 = struct.unpack("<4L", key)
    
    key0 = 0x51D067C
    key1 = 0x3F113D44
    key2 = 0x6AA301C0
    key3 = 0x72656277
    
    for i in range(32):
        v1 = v1 - ((v0 + sum_value) ^ (key2 + (v0 << 4)) ^ (key3 + (v0 >> 5)))
        v1 = uint32(v1)
        # print("v1:", hex(v1))
        v7 = uint32(v1 + sum_value)
        sum_value = uint32(sum_value + delta)
        v8 = v7 ^ uint32(key0 + (v1 << 4)) ^ uint32(key1 + (v1 >> 5))
        v0 =  uint32(v0 - v8)
        
    
    return struct.pack("<2L", v0, v1)



print(decrypt_block(c221, c222))

print(ord('\x87'))
print(ord('\xb5'))
print(ord('/'))
print(ord('_'))

import struct

print(struct.unpack('>H',b'3\xa3')[0])

b'3\xa3\x00\x00\x00\x00\x00\x00'
135
181
47
95
13219


## Sample ID

The new samples share common bytes related to the decryption algorithm `c1 e8 05 05 ?? ?? ?? ??33 c8 8b c3 c1 e0 04 05` which can be used to sig them.


```c
C1 E8 05                                shr     eax, 5
05 C0 36 F0 08                          add     eax, key3
33 C8                                   xor     ecx, eax
8B C3                                   mov     eax, ebx
C1 E0 04                                shl     eax, 4
05 85 49 77 4C                          add     eax, key2
33 C8                                   xor     ecx, eax
2B F1                                   sub     esi, ecx
8B C6                                   mov     eax, esi
C1 E0 04                                shl     eax, 4
05 54 55 93 7D                          add     eax, key0
8D 0C 37                                lea     ecx, [edi+esi]
33 C8                                   xor     ecx, eax
8D BF 47 86 C8 61                       lea     edi, [edi+61C88647h]
8B C6                                   mov     eax, esi
C1 E8 05                                shr     eax, 5
05 39 06 3B 7A                          add     eax, key1
33 C8                                   xor     ecx, eax
2B D9                                   sub     ebx, ecx
83 ED 01                                sub     ebp, 1
```




In [28]:
import re
import struct 

file_data = open('/tmp/mystic/mys.bin','rb').read()


egg = rb'\xc1\xe8\x05\x05(....)\x33\xc8\x8b\xc3\xc1\xe0\x04\x05(....)\x33\xc8\x2b\xf1\x8b\xc6\xc1\xe0\x04\x05(....)\x8d\x0c\x37\x33\xc8\x8d\xbf\x47\x86\xc8\x61\x8b\xc6\xc1\xe8\x05\x05(....)\x33\xc8\x2b\xd9\x83\xed\x01'


match = re.search(egg, file_data)


assert match is not None

key0 = struct.unpack('<I', match.group(3))[0]
key1 = struct.unpack('<I', match.group(4))[0]
key2 = struct.unpack('<I', match.group(2))[0]
key3 = struct.unpack('<I', match.group(1))[0]

print(f"{hex(key0)}\n{hex(key1)}\n{hex(key2)}\n{hex(key3)}\n")


def decrypt(data, key0, key1, key2, key3):
    out = b''
    block_size = 8  
    num_blocks = len(data) // block_size
    blocks = struct.unpack(f"<{2 * num_blocks}L", data)

    for i in range(0,len(blocks),2):
        out += decrypt_block(blocks[i],blocks[i+1], key0, key1, key2, key3)
    return out


def decrypt_block(v0, v1, key0, key1, key2, key3):
    sum_value = 0xC6EF3720
    delta = 0x61C88647
    
    #key0, key1, key2, key3 = struct.unpack("<4L", key)
    
    for i in range(32):
        v1 = v1 - ((v0 + sum_value) ^ (key2 + (v0 << 4)) ^ (key3 + (v0 >> 5)))
        v1 = uint32(v1)
        # print("v1:", hex(v1))
        v7 = uint32(v1 + sum_value)
        sum_value = uint32(sum_value + delta)
        v8 = v7 ^ uint32(key0 + (v1 << 4)) ^ uint32(key1 + (v1 >> 5))
        v0 =  uint32(v0 - v8)
        
    
    return struct.pack("<2L", v0, v1)

0x7d935554
0x7a3b0639
0x4c774985
0x8f036c0



In [34]:
import pefile

pe = pefile.PE(data=file_data)

rdata = None 

for s in pe.sections:
    if s.Name[:6] == b'.rdata':
        rdata = s.get_data()
        
assert rdata is not None


candidates = rdata.split(b'\x00\x00\x00')

found = False
for c in candidates:
    if found:
        break
    c = c.strip(b'\x00').rstrip(b'\x00')
    
    if len(c) < 11:
        continue
    
    for i in range(len(c) - 10):
        tmp = c[i:]
        #print(f'\n\nTesting: {tmp}')
        try:
            out = decrypt(tmp, key0, key1, key2, key3)

            if out[:4] == b'http':
                print(out)
                found = True
                break
        except:
            pass

        

b'http://193.233.254.61/\x00\x00'


### Config Extraction 

In [37]:
import re
import struct
import pefile




def extract(file_path):
    file_data = open(file_path,'rb').read()


    egg = rb'\xc1\xe8\x05\x05(....)\x33\xc8\x8b\xc3\xc1\xe0\x04\x05(....)\x33\xc8\x2b\xf1\x8b\xc6\xc1\xe0\x04\x05(....)\x8d\x0c\x37\x33\xc8\x8d\xbf\x47\x86\xc8\x61\x8b\xc6\xc1\xe8\x05\x05(....)\x33\xc8\x2b\xd9\x83\xed\x01'


    match = re.search(egg, file_data)


    assert match is not None

    key0 = struct.unpack('<I', match.group(3))[0]
    key1 = struct.unpack('<I', match.group(4))[0]
    key2 = struct.unpack('<I', match.group(2))[0]
    key3 = struct.unpack('<I', match.group(1))[0]
    
    
    
    pe = pefile.PE(data=file_data)

    rdata = None 

    for s in pe.sections:
        if s.Name[:6] == b'.rdata':
            rdata = s.get_data()

    assert rdata is not None


    candidates = rdata.split(b'\x00\x00\x00')

    found = False
    for c in candidates:
        if found:
            break
        c = c.strip(b'\x00').rstrip(b'\x00')

        if len(c) < 11:
            continue

        for i in range(len(c) - 10):
            tmp = c[i:]
            #print(f'\n\nTesting: {tmp}')
            try:
                out = decrypt(tmp, key0, key1, key2, key3)

                if out[:4] == b'http':
                    print(out)
                    found = True
                    break
            except:
                pass

            
extract('/tmp/mystic/mys.bin')


b'http://193.233.254.61/\x00\x00'


In [40]:
import os
# assign directory
directory = '/tmp/test/'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        try:
            print(f)
            extract(f)
        except:
            pass

/tmp/test/e59b04adba48e393f70f266963502d0874ab4922d82771d34201c5437f13fa1d
b'http://193.233.254.61/\x00\x00'
/tmp/test/57b5d8f8ac46c06e4557306f1695d84ac69abd4fe0c7483c0f629c759a19af84
b'http://193.233.254.61/\x00\x00'
/tmp/test/0f25abd7883b4fde66a267efdc2b81276660e4e086609f1e463eaf148a845412
b'http://193.233.254.61/\x00\x00'
/tmp/test/9dd4e6ebfed11febb02847a221bdcf7067954e2a5c08469fa6cf9a125c13bff1
b'http://193.233.254.61/\x00\x00'
/tmp/test/d834a9551db2db0f36b8e0b38f65a8caa134b1133ba5490359edb04679eb5db6
b'http://193.233.254.61/\x00\x00'
/tmp/test/9960e5aa85b8d62c322ff29ebd9cfbff638975fedffd1c288027e5e9d5743f26
b'http://193.233.254.61/\x00\x00'
/tmp/test/f78d0fc08a32b469f4a12b12ec350e0d9cb6441ad8d3a16eeaf1af9194c16e60
/tmp/test/f59fe5f95fdfa3dbd342360eaf42495e3a51d4dae1ff4a7ee2b38a7a5daaf0fb
b'http://193.233.254.61/\x00\x00'
/tmp/test/0326eb043f804b7930c136027f9e5c0244891d1501ad13a80da06548c2e1a3d9
b'http://193.233.254.61/\x00\x00'
/tmp/test/9c1c2cd81b3e62965190e8656e1b3c45e0d8ae3d355